In [121]:
import pandas as pd
import nltk
import numpy as np
import tqdm
import re
import math

In [101]:
all_answers = pd.read_csv('./data/all_shuff_answers.tsv', sep='\t', header=None)

In [130]:
ask_status = ['Новый', 'Инцидент']

def norm(text):
    text = text.replace('\t', ' ').replace('\n', ' ').replace('\'', '').replace('<br>', ' ')
    text = re.sub(r'[^A-Za-zА-Яа-я0-9\s]+', ' ', text)
    return text[:500]

with open('./data/all_data.tsv', 'w') as fd:
    for df in tqdm.tqdm(pd.read_csv('./data/dialogs_1.csv', chunksize=10000)):
        df = df.dropna(subset=['Контент', 'ID'])
        pos_pair = []
        
        num_comments = 0
        total = 0
        
        for idx0, (key, group) in enumerate(df.groupby('ID')):
            
            total += 1
            
            if (group['Тип.обращения'] == 'Комментарий').sum() > 0:
                continue
                        
            question = []
            answer = None
            for row in group.itertuples():
                if not pd.isnull(row[8]):
                    question.append(row.Контент)
                else:
                    answer = row.Контент
                    break
            if answer:
                question = norm(' '.join(question))
                answer = norm(answer.replace('\t', ' '))
                
                fd.write("{}\t{}\t{}\n".format(question, answer, 1))
                
                pos_pair.append((question, answer))

                for idx in range(10):
                    rand_answer = np.random.choice(all_answers[0])
                    fd.write("{}\t{}\t{}\n".format(question, norm(rand_answer), 0))
#             if idx0 > 3:
#                 break
#         break

3it [00:07,  2.66s/it]/home/generall/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
95it [02:47,  1.77s/it]


In [131]:
!wc -l ./data/all_data.tsv

389279 ./data/all_data.tsv


In [138]:
!shuf ./data/all_data.tsv > ./data/all_data_shuf.tsv
!mv ./data/all_data_shuf.tsv ./data/all_data.tsv

In [139]:
valid_size = 50000

In [140]:
train_size = 389279 - valid_size

In [141]:
!tail -n $valid_size ./data/all_data.tsv > ./data/valid_data.tsv

In [142]:
!head -n $train_size ./data/all_data.tsv > ./data/train_data.tsv

In [113]:
6630 * 50

331500

In [146]:
df = pd.read_csv('./data/valid_data.tsv', sep='\t', nrows=10000, header=None)

In [148]:
df[df[2] == 0][[0, 1]].head(10).to_csv('./data/neg_example.tsv', sep='\t', index=None, header=None)